Predictive question: 
Using CPI, Current account, and unemployment rate to predict Real GNI of 2020, 2021, 2022 
using regression analysis 2013-2019

Response variable (Dependent): Real GNI 

Model: regression

Variables:

Real GNI, annually, Canada, 2013-2022
gni.csv

CPI, unadjusted, 2013-2022
cpi.csv

Current account, annual, Canada, 2013-2022
current_account.csv

Unemployment rate, age 15 or above, Canada, both sexes, 2013-2022
unemployment.csv

Things to complete in our FINAL PROJECT

- library
- Read and load the files
- merge data
- regression analysis (cross-validation, standard error, prediction, visualization)
- written report on what the processes are
- results
- implications

